# User Interface

In [3]:
# Import necessary libraries
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import json
import random
import re
import requests
import os
import sys

# Global data storage
app_data = {
    "atm": {"pin": None, "balance": 0, "transactions": []},
    "todo": [],
    "grade_tracker": {"students": []},
    "rock_paper_scissors": {"user_score": 0, "computer_score": 0},
    "api_keys": {
        "weather": "",
        "currency": ""
    }
}

# -------------------------
# Helper Functions
# -------------------------

def load_atm_data():
    try:
        with open("atm_data.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return {"pin": None, "balance": 0, "transactions": []}

def save_atm_data(data):
    with open("atm_data.json", "w") as file:
        json.dump(data, file)

def load_todo_data():
    try:
        with open("tasks.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return []

def save_todo_data(tasks):
    with open("tasks.json", "w") as file:
        json.dump(tasks, file)

# -------------------------
# Application Widgets
# -------------------------

# Main application selector
app_selector = widgets.Dropdown(
    options=[
        'Select an Application',
        'ATM Machine', 
        'Calculator', 
        'Currency Exchanger',
        'Even/Odd Checker',
        'Grade Tracker',
        'Multiplication Table',
        'Number Guessing Game',
        'Palindrome Checker',
        'Password Checker',
        'Rock Paper Scissors',
        'To-Do List',
        'Weather App',
        'Word Counter',
        'Wordle Game'
    ],
    value='Select an Application',
    description='App:',
    style={'description_width': 'initial'}
)

# Main content area
content_area = widgets.Output()

# Application header
app_header = widgets.HTML(value="<h1>Python Applications</h1><p>Select an application from the dropdown above to get started.</p>")

# -------------------------
# ATM Application
# -------------------------

def create_atm_app():
    # Load existing data
    atm_data = load_atm_data()
    
    # ATM header
    atm_header = widgets.HTML(value="<h2>ATM Machine</h2>")
    
    # PIN section
    pin_input = widgets.Password(description="PIN:", placeholder="Enter 4-digit PIN")
    pin_message = widgets.HTML(value="")
    
    # Operation selector
    operation_selector = widgets.Dropdown(
        options=['Select Operation', 'Check Balance', 'Withdraw', 'Deposit', 'Change PIN'],
        value='Select Operation',
        description='Operation:',
        style={'description_width': 'initial'}
    )
    
    # Dynamic content area for ATM operations
    atm_content = widgets.VBox([])
    
    # Create PIN if not exists
    def check_pin_status():
        nonlocal atm_data
        if atm_data["pin"] is None:
            pin_message.value = "<p style='color:blue'>Please create a new 4-digit PIN</p>"
        else:
            pin_message.value = "<p>Enter your PIN to continue</p>"
    
    # Verify or create PIN
    def on_pin_submit(b):
        nonlocal atm_data
        pin = pin_input.value
        
        if not pin.isdigit() or len(pin) != 4:
            pin_message.value = "<p style='color:red'>PIN must be exactly 4 digits</p>"
            return
            
        if atm_data["pin"] is None:
            # Creating new PIN
            atm_data["pin"] = pin
            save_atm_data(atm_data)
            pin_message.value = "<p style='color:green'>PIN created successfully!</p>"
            pin_input.value = ""  # Clear for security
            # Show operations after creating PIN
            atm_container.children = [atm_header, operation_selector, atm_content]
        else:
            # Verifying existing PIN
            if pin == atm_data["pin"]:
                pin_message.value = "<p style='color:green'>PIN verified successfully!</p>"
                pin_input.value = ""  # Clear for security
                # Show operations after verifying PIN
                atm_container.children = [atm_header, operation_selector, atm_content]
            else:
                pin_message.value = "<p style='color:red'>Invalid PIN. Please try again.</p>"
    
    # PIN submission button
    pin_button = widgets.Button(description="Submit PIN")
    pin_button.on_click(on_pin_submit)
    
    # PIN section container
    pin_section = widgets.VBox([pin_input, pin_button, pin_message])
    
    # Handle ATM operations
    def on_operation_change(change):
        operation = change['new']
        atm_content.children = []
        
        if operation == 'Check Balance':
            balance_widget = widgets.HTML(
                value=f"<h3>Current Balance: ${atm_data['balance']:.2f}</h3>"
            )
            atm_content.children = [balance_widget]
            
        elif operation == 'Withdraw':
            amount_input = widgets.FloatText(description='Amount: $', value=0)
            result_widget = widgets.HTML(value="")
            
            def on_withdraw(b):
                amount = amount_input.value
                if amount <= 0:
                    result_widget.value = "<p style='color:red'>Amount must be greater than zero</p>"
                elif amount > atm_data['balance']:
                    result_widget.value = f"<p style='color:red'>Insufficient funds. Available balance: ${atm_data['balance']:.2f}</p>"
                else:
                    atm_data['balance'] -= amount
                    atm_data['transactions'].append(f"Withdrawal: -${amount:.2f}")
                    save_atm_data(atm_data)
                    result_widget.value = f"<p style='color:green'>Successfully withdrew ${amount:.2f}. New balance: ${atm_data['balance']:.2f}</p>"
                    amount_input.value = 0
            
            withdraw_button = widgets.Button(description="Withdraw")
            withdraw_button.on_click(on_withdraw)
            
            atm_content.children = [amount_input, withdraw_button, result_widget]
            
        elif operation == 'Deposit':
            amount_input = widgets.FloatText(description='Amount: $', value=0)
            result_widget = widgets.HTML(value="")
            
            def on_deposit(b):
                amount = amount_input.value
                if amount <= 0:
                    result_widget.value = "<p style='color:red'>Amount must be greater than zero</p>"
                else:
                    atm_data['balance'] += amount
                    atm_data['transactions'].append(f"Deposit: +${amount:.2f}")
                    save_atm_data(atm_data)
                    result_widget.value = f"<p style='color:green'>Successfully deposited ${amount:.2f}. New balance: ${atm_data['balance']:.2f}</p>"
                    amount_input.value = 0
            
            deposit_button = widgets.Button(description="Deposit")
            deposit_button.on_click(on_deposit)
            
            atm_content.children = [amount_input, deposit_button, result_widget]
            
        elif operation == 'Change PIN':
            new_pin_input = widgets.Password(description="New PIN:", placeholder="Enter 4-digit PIN")
            result_widget = widgets.HTML(value="")
            
            def on_change_pin(b):
                new_pin = new_pin_input.value
                if not new_pin.isdigit() or len(new_pin) != 4:
                    result_widget.value = "<p style='color:red'>PIN must be exactly 4 digits</p>"
                else:
                    atm_data['pin'] = new_pin
                    save_atm_data(atm_data)
                    result_widget.value = "<p style='color:green'>PIN changed successfully!</p>"
                    new_pin_input.value = ""  # Clear for security
            
            change_pin_button = widgets.Button(description="Change PIN")
            change_pin_button.on_click(on_change_pin)
            
            atm_content.children = [new_pin_input, change_pin_button, result_widget]
    
    # Connect operation selector to handler
    operation_selector.observe(on_operation_change, names='value')
    
    # ATM container
    atm_container = widgets.VBox([atm_header, pin_section])
    
    # Initialize PIN status
    check_pin_status()
    
    return atm_container

# -------------------------
# Calculator Application
# -------------------------

def create_calculator_app():
    calc_header = widgets.HTML(value="<h2>Simple Calculator</h2>")
    
    # Display area
    display_widget = widgets.HTML(value="<h3>0</h3>")
    
    # Number input
    number_input = widgets.FloatText(description='Input:', value=0)
    
    # Current value storage
    calc_data = {'current_value': 0, 'operation': None}
    
    # Operation buttons
    def create_op_button(op, symbol):
        button = widgets.Button(description=symbol, layout=widgets.Layout(width='50px'))
        
        def on_click(b):
            calc_data['current_value'] = number_input.value
            calc_data['operation'] = op
            display_widget.value = f"<h3>{calc_data['current_value']} {symbol}</h3>"
            number_input.value = 0
            
        button.on_click(on_click)
        return button
    
    # Calculate button
    equals_button = widgets.Button(description="=", layout=widgets.Layout(width='50px'))
    
    def on_equals(b):
        second_value = number_input.value
        if calc_data['operation'] == '+':
            result = calc_data['current_value'] + second_value
        elif calc_data['operation'] == '-':
            result = calc_data['current_value'] - second_value
        elif calc_data['operation'] == '*':
            result = calc_data['current_value'] * second_value
        elif calc_data['operation'] == '/':
            if second_value == 0:
                display_widget.value = "<h3 style='color:red'>Error: Division by zero</h3>"
                return
            result = calc_data['current_value'] / second_value
        else:
            result = second_value
            
        display_widget.value = f"<h3>{result}</h3>"
        number_input.value = result
        calc_data['current_value'] = result
        calc_data['operation'] = None
        
    equals_button.on_click(on_equals)
    
    # Clear button
    clear_button = widgets.Button(description="C", layout=widgets.Layout(width='50px'))
    
    def on_clear(b):
        calc_data['current_value'] = 0
        calc_data['operation'] = None
        display_widget.value = "<h3>0</h3>"
        number_input.value = 0
        
    clear_button.on_click(on_clear)
    
    # Button layout
    op_buttons = widgets.HBox([
        create_op_button('+', '+'),
        create_op_button('-', '-'),
        create_op_button('*', '×'),
        create_op_button('/', '÷')
    ])
    
    control_buttons = widgets.HBox([equals_button, clear_button])
    
    calculator_container = widgets.VBox([
        calc_header,
        display_widget,
        number_input,
        op_buttons,
        control_buttons
    ])
    
    return calculator_container

# -------------------------
# Currency Exchanger
# -------------------------

def create_currency_exchanger():
    currency_header = widgets.HTML(value="<h2>Currency Exchanger</h2>")
    
    # Input fields
    api_key_input = widgets.Password(
        description='API Key:',
        placeholder='Enter your API key from currencyapi.com',
        style={'description_width': 'initial'},
        value=app_data["api_keys"]["currency"]
    )
    
    from_currency = widgets.Text(
        description='From:',
        placeholder='USD',
        value='USD'
    )
    
    to_currency = widgets.Text(
        description='To:',
        placeholder='EUR',
        value='EUR'
    )
    
    amount_input = widgets.FloatText(
        description='Amount:',
        value=1.0
    )
    
    result_area = widgets.HTML(value="")
    
    # Convert button
    convert_button = widgets.Button(description="Convert")
    
    def on_convert(b):
        api_key = api_key_input.value
        # Save the API key to the global data
        app_data["api_keys"]["currency"] = api_key
        
        if not api_key:
            result_area.value = "<p style='color:red'>Please enter an API key from currencyapi.com</p>"
            return
            
        from_curr = from_currency.value.upper()
        to_curr = to_currency.value.upper()
        amount = amount_input.value
        
        url = f"https://api.currencyapi.com/v3/latest?apikey={api_key}&base_currency={from_curr}&currencies={to_curr}"
        
        try:
            result_area.value = "<p>Fetching exchange rates...</p>"
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            exchange_rate = data["data"][to_curr]["value"]
            converted_amount = amount * exchange_rate
            
            result_area.value = f"""
            <div style='padding:10px; background-color:#f0f8ff; border-radius:5px;'>
                <p><b>Exchange Rate:</b> 1 {from_curr} = {exchange_rate} {to_curr}</p>
                <p><b>Converted Amount:</b> {amount} {from_curr} = {converted_amount:.2f} {to_curr}</p>
            </div>
            """
        except requests.exceptions.RequestException as e:
            result_area.value = f"<p style='color:red'>Error fetching currency data: {str(e)}</p>"
        except KeyError:
            result_area.value = "<p style='color:red'>Invalid currency code. Please check and try again.</p>"
    
    convert_button.on_click(on_convert)
    
    currency_container = widgets.VBox([
        currency_header,
        widgets.HTML(value="<p>You need an API key from <a href='https://app.currencyapi.com' target='_blank'>currencyapi.com</a></p>"),
        api_key_input,
        from_currency,
        to_currency,
        amount_input,
        convert_button,
        result_area
    ])
    
    return currency_container

# -------------------------
# Even/Odd Checker
# -------------------------

def create_even_odd_checker():
    header = widgets.HTML(value="<h2>Even/Odd Number Checker</h2>")
    
    number_input = widgets.IntText(description='Number:')
    result_area = widgets.HTML(value="")
    
    check_button = widgets.Button(description="Check")
    
    def on_check(b):
        try:
            number = number_input.value
            if number % 2 == 0:
                result_area.value = f"<p style='color:blue'>{number} is <b>Even</b></p>"
            else:
                result_area.value = f"<p style='color:green'>{number} is <b>Odd</b></p>"
        except Exception as e:
            result_area.value = f"<p style='color:red'>Error: {str(e)}</p>"
    
    check_button.on_click(on_check)
    
    container = widgets.VBox([
        header,
        number_input,
        check_button,
        result_area
    ])
    
    return container

# -------------------------
# Grade Tracker
# -------------------------

def create_grade_tracker():
    header = widgets.HTML(value="<h2>Student Grade Tracker</h2>")
    
    # Check if grade tracker data exists
    if "students" not in app_data["grade_tracker"]:
        app_data["grade_tracker"]["students"] = []
    
    # Input fields for adding students
    name_input = widgets.Text(description='Name:')
    grade_input = widgets.FloatText(description='Grade:')
    
    result_area = widgets.HTML(value="")
    
    # Display area for student list and stats
    students_display = widgets.HTML(value="<p>No students added yet.</p>")
    
    # Add student button
    add_button = widgets.Button(description="Add Student")
    
    def on_add_student(b):
        try:
            name = name_input.value
            grade = grade_input.value
            
            if not name:
                result_area.value = "<p style='color:red'>Please enter a student name</p>"
                return
                
            app_data["grade_tracker"]["students"].append({"name": name, "grade": grade})
            name_input.value = ""
            grade_input.value = 0
            
            result_area.value = f"<p style='color:green'>Added {name} with grade {grade}</p>"
            update_students_display()
        except Exception as e:
            result_area.value = f"<p style='color:red'>Error: {str(e)}</p>"
    
    add_button.on_click(on_add_student)
    
    # Calculate average button
    calc_button = widgets.Button(description="Calculate Average")
    
    def on_calculate(b):
        students = app_data["grade_tracker"]["students"]
        
        if not students:
            result_area.value = "<p style='color:orange'>No students added yet.</p>"
            return
            
        total_grade = sum(student["grade"] for student in students)
        average_grade = total_grade / len(students)
        
        result_area.value = f"""
        <div style='padding:10px; background-color:#f0f8ff; border-radius:5px;'>
            <p><b>Total Students:</b> {len(students)}</p>
            <p><b>Average Grade:</b> {average_grade:.2f}</p>
        </div>
        """
    
    calc_button.on_click(on_calculate)
    
    # Function to update the students display
    def update_students_display():
        students = app_data["grade_tracker"]["students"]
        
        if not students:
            students_display.value = "<p>No students added yet.</p>"
            return
            
        html = "<h3>Student List</h3><table style='width:100%; border-collapse:collapse;'>"
        html += "<tr><th style='border:1px solid #ddd; padding:8px;'>Name</th><th style='border:1px solid #ddd; padding:8px;'>Grade</th></tr>"
        
        for student in students:
            html += f"<tr><td style='border:1px solid #ddd; padding:8px;'>{student['name']}</td><td style='border:1px solid #ddd; padding:8px;'>{student['grade']}</td></tr>"
            
        html += "</table>"
        students_display.value = html
    
    # Clear button
    clear_button = widgets.Button(description="Clear All")
    
    def on_clear(b):
        app_data["grade_tracker"]["students"] = []
        result_area.value = "<p>All student data cleared</p>"
        update_students_display()
    
    clear_button.on_click(on_clear)
    
    # Update display on initial load
    update_students_display()
    
    # Buttons layout
    buttons = widgets.HBox([add_button, calc_button, clear_button])
    
    container = widgets.VBox([
        header,
        widgets.HBox([name_input, grade_input]),
        buttons,
        result_area,
        students_display
    ])
    
    return container

# -------------------------
# Multiplication Table
# -------------------------

def create_multiplication_table():
    header = widgets.HTML(value="<h2>Multiplication Table</h2>")
    
    number_input = widgets.IntText(description='Number:')
    result_area = widgets.HTML(value="")
    
    generate_button = widgets.Button(description="Generate Table")
    
    def on_generate(b):
        try:
            number = number_input.value
            
            html = f"<h3>Multiplication Table for {number}</h3>"
            html += "<table style='width:300px; border-collapse:collapse;'>"
            
            for i in range(1, 11):
                result = number * i
                html += f"<tr><td style='border:1px solid #ddd; padding:8px;'>{number} × {i}</td><td style='border:1px solid #ddd; padding:8px;'>{result}</td></tr>"
                
            html += "</table>"
            
            result_area.value = html
        except Exception as e:
            result_area.value = f"<p style='color:red'>Error: {str(e)}</p>"
    
    generate_button.on_click(on_generate)
    
    container = widgets.VBox([
        header,
        number_input,
        generate_button,
        result_area
    ])
    
    return container

# -------------------------
# Number Guessing Game
# -------------------------

def create_number_guessing_game():
    header = widgets.HTML(value="<h2>Number Guessing Game</h2>")
    
    # Setup inputs
    min_input = widgets.IntText(description='Min:', value=1)
    max_input = widgets.IntText(description='Max:', value=100)
    
    # Game status and controls
    game_status = widgets.HTML(value="<p>Set your range and start a new game</p>")
    guess_input = widgets.IntText(description='Guess:')
    
    # Game data
    game_data = {'target': None, 'attempts': 0, 'min': 1, 'max': 100, 'game_active': False}
    
    # Start game button
    start_button = widgets.Button(description="Start New Game")
    
    def on_start_game(b):
        try:
            game_data['min'] = min_input.value
            game_data['max'] = max_input.value
            
            if game_data['min'] >= game_data['max']:
                game_status.value = "<p style='color:red'>Error: Min must be less than Max</p>"
                return
                
            game_data['target'] = random.randint(game_data['min'], game_data['max'])
            game_data['attempts'] = 0
            game_data['game_active'] = True
            
            game_status.value = f"<p>I'm thinking of a number between {game_data['min']} and {game_data['max']}. Make a guess!</p>"
            guess_input.min = game_data['min']
            guess_input.max = game_data['max']
        except Exception as e:
            game_status.value = f"<p style='color:red'>Error: {str(e)}</p>"
    
    start_button.on_click(on_start_game)
    
    # Guess button
    guess_button = widgets.Button(description="Submit Guess")
    
    def on_guess(b):
        if not game_data['game_active']:
            game_status.value = "<p style='color:orange'>Please start a new game first</p>"
            return
            
        guess = guess_input.value
        game_data['attempts'] += 1
        
        if guess < game_data['target']:
            game_status.value = f"<p style='color:blue'>Too low! Try again. (Attempts: {game_data['attempts']})</p>"
        elif guess > game_data['target']:
            game_status.value = f"<p style='color:blue'>Too high! Try again. (Attempts: {game_data['attempts']})</p>"
        else:
            game_status.value = f"<p style='color:green'>Congratulations! You guessed the number {game_data['target']} in {game_data['attempts']} attempts!</p>"
            game_data['game_active'] = False
    
    guess_button.on_click(on_guess)
    
    container = widgets.VBox([
        header,
        widgets.HBox([min_input, max_input]),
        start_button,
        game_status,
        widgets.HBox([guess_input, guess_button])
    ])
    
    return container

# -------------------------
# Palindrome Checker
# -------------------------

def create_palindrome_checker():
    header = widgets.HTML(value="<h2>Palindrome Checker</h2>")
    
    text_input = widgets.Text(
        description='Text:',
        placeholder='Enter a word or phrase',
        style={'description_width': 'initial'}
    )
    
    result_area = widgets.HTML(value="")
    
    check_button = widgets.Button(description="Check")
    
    def on_check(b):
        text = text_input.value
        
        if not text:
            result_area.value = "<p style='color:orange'>Please enter some text</p>"
            return
            
        # Remove spaces and convert to lowercase
        processed_text = text.lower().replace(" ", "")
        
        if processed_text == processed_text[::-1]:
            result_area.value = f"<p style='color:green'>'{text}' is a palindrome!</p>"
        else:
            result_area.value = f"<p style='color:red'>'{text}' is not a palindrome.</p>"
    
    check_button.on_click(on_check)
    
    container = widgets.VBox([
        header,
        text_input,
        check_button,
        result_area
    ])
    
    return container

# -------------------------
# Password Checker
# -------------------------

def create_password_checker():
    header = widgets.HTML(value="<h2>Password Strength Checker</h2>")
    
    password_input = widgets.Password(
        description='Password:',
        placeholder='Enter a password'
    )
    
    result_area = widgets.HTML(value="")
    
    check_button = widgets.Button(description="Check Strength")
    
    def on_check(b):
        password = password_input.value
        
        if not password:
            result_area.value = "<p style='color:orange'>Please enter a password</p>"
            return
            
        # Check password strength
        checks = []
        strength = "Strong"
        
        if len(password) < 8:
            checks.append("Password must be at least 8 characters long")
            strength = "Weak"
            
        if not re.search(r'[A-Z]', password):
            checks.append("Password must contain at least one uppercase letter")
            strength = "Weak"
            
        if not re.search(r'[a-z]', password):
            checks.append("Password must contain at least one lowercase letter")
            strength = "Weak"
            
        if not re.search(r'\d', password):
            checks.append("Password must contain at least one number")
            strength = "Weak"
            
        if not re.search(r'[!@#$%^&*(),.?":{}|<>]', password):
            checks.append("Password must contain at least one special character")
            strength = "Weak"
            
        # Build result HTML
        if strength == "Strong":
            result_area.value = "<p style='color:green'><b>Strong Password!</b> Your password meets all security requirements.</p>"
        else:
            html = "<p style='color:red'><b>Weak Password!</b> Please address the following issues:</p><ul>"
            for check in checks:
                html += f"<li>{check}</li>"
            html += "</ul>"
            result_area.value = html
    
    check_button.on_click(on_check)
    
    container = widgets.VBox([
        header,
        password_input,
        check_button,
        result_area
    ])
    
    return container

# -------------------------
# Rock Paper Scissors
# -------------------------

def create_rock_paper_scissors():
    header = widgets.HTML(value="<h2>Rock Paper Scissors</h2>")
    
    # Game data
    game_data = app_data["rock_paper_scissors"]
    
    # Game controls
    choice_buttons = widgets.HBox([
        widgets.Button(description="Rock", layout=widgets.Layout(width='100px')),
        widgets.Button(description="Paper", layout=widgets.Layout(width='100px')),
        widgets.Button(description="Scissors", layout=widgets.Layout(width='100px'))
    ])
    
    # Result displays
    game_result = widgets.HTML(value="<p>Make your choice!</p>")
    score_display = widgets.HTML(
        value=f"<p><b>Score:</b> You: {game_data['user_score']} | Computer: {game_data['computer_score']}</p>"
    )
    
    # Handle player choice
    def on_choice(b):
        user_choice = b.description.lower()
        computer_choice = random.choice(["rock", "paper", "scissors"])
        
        # Determine winner
        if user_choice == computer_choice:
            result = f"It's a tie! Computer: {computer_choice}, You: {user_choice}"
            color = "blue"
        elif (user_choice == "rock" and computer_choice == "scissors") or \
             (user_choice == "paper" and computer_choice == "rock") or \
             (user_choice == "scissors" and computer_choice == "paper"):
            result = f"You win! Computer: {computer_choice}, You: {user_choice}"
            color = "green"
            game_data['user_score'] += 1
        else:
            result = f"Computer wins! Computer: {computer_choice}, You: {user_choice}"
            color = "red"
            game_data['computer_score'] += 1
            
        game_result.value = f"<p style='color:{color}'>{result}</p>"
        score_display.value = f"<p><b>Score:</b> You: {game_data['user_score']} | Computer: {game_data['computer_score']}</p>"
    
    for button in choice_buttons.children:
        button.on_click(on_choice)
    
    # Reset button
    reset_button = widgets.Button(description="Reset Score")
    
    def on_reset(b):
        game_data['user_score'] = 0
        game_data['computer_score'] = 0
        score_display.value = f"<p><b>Score:</b> You: 0 | Computer: 0</p>"
        game_result.value = "<p>Score reset! Make your choice.</p>"
    
    reset_button.on_click(on_reset)
    
    container = widgets.VBox([
        header,
        score_display,
        choice_buttons,
        reset_button,
        game_result
    ])
    
    return container

# -------------------------
# To-Do List
# -------------------------

def create_todo_list():
    header = widgets.HTML(value="<h2>To-Do List</h2>")
    
    # Load existing tasks
    tasks = load_todo_data()
    
    # Input field for new tasks
    task_input = widgets.Text(
        description='Task:',
        placeholder='Enter a new task'
    )
    
    # Task display area
    tasks_display = widgets.HTML(value="")
    
    # Result message area
    result_area = widgets.HTML(value="")
    
    # Function to update task display
    def update_tasks_display():
        if not tasks:
            tasks_display.value = "<p>No tasks yet. Add some tasks to get started!</p>"
            return
            
        html = "<h3>Your Tasks</h3><ul style='list-style-type:none; padding-left:0;'>"
        
        for i, task in enumerate(tasks):
            status = "✅" if task["completed"] else "⬜"
            task_text = f"<span style='text-decoration: line-through;'>{task['task']}</span>" if task["completed"] else task['task']
            # Added unique IDs to links to avoid potential conflicts
            html += f"<li style='margin-bottom:8px;'>{status} {task_text} "
            
            if not task["completed"]:
                html += f"[<a href='#' id='complete-{i}' data-action='complete' data-index='{i}'>Complete</a>] "
            
            html += f"[<a href='#' id='delete-{i}' data-action='delete' data-index='{i}'>Delete</a>]</li>"
            
        html += "</ul>"
        
        # Improved JavaScript to handle task actions with unique selectors and error handling
        html += """
        <script>
        (function() {
            // Wait for elements to be available
            setTimeout(function() {
                document.querySelectorAll('a[data-action]').forEach(link => {
                    link.addEventListener('click', function(e) {
                        e.preventDefault();
                        try {
                            const action = this.getAttribute('data-action');
                            const index = this.getAttribute('data-index');
                            // Execute command in Python
                            if (typeof IPython !== 'undefined' && IPython.notebook && IPython.notebook.kernel) {
                                const command = `handle_task_action('${action}', ${index})`;
                                IPython.notebook.kernel.execute(command);
                            } else {
                                console.error('IPython kernel not available');
                            }
                        } catch (error) {
                            console.error('Error handling task action:', error);
                        }
                    });
                });
            }, 100);
        })();
        </script>
        """
        
        tasks_display.value = html
    
    # Create a closure for task actions to capture the tasks variable
    def create_task_handler():
        # This is a closure that captures the tasks variable
        def handle_task_action(action, index):
            nonlocal tasks
            if action == 'complete':
                tasks[index]["completed"] = True
                result_area.value = "<p style='color:green'>Task marked as completed!</p>"
            elif action == 'delete':
                del tasks[index]
                result_area.value = "<p style='color:blue'>Task deleted!</p>"
                
            save_todo_data(tasks)
            update_tasks_display()
        return handle_task_action
    
    # Create the handler with access to the current tasks
    handle_task_action = create_task_handler()
    
    # Register the function in the global namespace so it can be called from JavaScript
    import IPython
    IPython.get_ipython().user_ns['handle_task_action'] = handle_task_action
    
    # Add task button
    add_button = widgets.Button(description="Add Task")
    
    def on_add_task(b):
        task_text = task_input.value
        
        if not task_text:
            result_area.value = "<p style='color:orange'>Please enter a task</p>"
            return
            
        tasks.append({"task": task_text, "completed": False})
        save_todo_data(tasks)
        task_input.value = ""
        result_area.value = "<p style='color:green'>Task added successfully!</p>"
        update_tasks_display()
    
    add_button.on_click(on_add_task)
    
    # Clear completed button
    clear_button = widgets.Button(description="Clear Completed")
    
    def on_clear_completed(b):
        nonlocal tasks
        tasks = [task for task in tasks if not task["completed"]]
        save_todo_data(tasks)
        result_area.value = "<p style='color:blue'>Completed tasks cleared!</p>"
        update_tasks_display()
    
    clear_button.on_click(on_clear_completed)
    
    # Initialize task display
    update_tasks_display()
    
    container = widgets.VBox([
        header,
        widgets.HBox([task_input, add_button]),
        clear_button,
        result_area,
        tasks_display
    ])
    
    return container

# -------------------------
# Weather App
# -------------------------

def create_weather_app():
    header = widgets.HTML(value="<h2>Weather Application</h2>")
    
    # Input fields
    api_key_input = widgets.Password(
        description='API Key:',
        placeholder='Enter your API key from weatherapi.com',
        style={'description_width': 'initial'},
        value=app_data["api_keys"]["weather"]
    )
    
    city_input = widgets.Text(
        description='City:',
        placeholder='Enter a city name'
    )
    
    result_area = widgets.HTML(value="")
    
    # Get weather button
    weather_button = widgets.Button(description="Get Weather")
    
    def on_get_weather(b):
        api_key = api_key_input.value
        city = city_input.value
        
        # Save the API key to the global data
        app_data["api_keys"]["weather"] = api_key
        
        if not api_key:
            result_area.value = "<p style='color:red'>Please enter an API key from weatherapi.com</p>"
            return
            
        if not city:
            result_area.value = "<p style='color:orange'>Please enter a city name</p>"
            return
            
        url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"
        
        try:
            result_area.value = "<p>Fetching weather data...</p>"
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            location = data["location"]["name"]
            temperature = data["current"]["temp_c"]
            condition = data["current"]["condition"]["text"]
            humidity = data["current"]["humidity"]
            wind_speed = data["current"]["wind_kph"]
            feels_like = data["current"]["feelslike_c"]
            
            result_area.value = f"""
            <div style='padding:15px; background-color:#f0f8ff; border-radius:8px;'>
                <h3>Weather in {location}</h3>
                <p><b>Temperature:</b> {temperature}°C (Feels like: {feels_like}°C)</p>
                <p><b>Condition:</b> {condition}</p>
                <p><b>Humidity:</b> {humidity}%</p>
                <p><b>Wind Speed:</b> {wind_speed} km/h</p>
            </div>
            """
        except requests.exceptions.RequestException as e:
            result_area.value = f"<p style='color:red'>Error fetching weather data: {str(e)}</p>"
        except (KeyError, ValueError) as e:
            result_area.value = f"<p style='color:red'>Error processing weather data: {str(e)}</p>"
    
    weather_button.on_click(on_get_weather)
    
    container = widgets.VBox([
        header,
        widgets.HTML(value="<p>You need an API key from <a href='https://www.weatherapi.com' target='_blank'>weatherapi.com</a></p>"),
        api_key_input,
        city_input,
        weather_button,
        result_area
    ])
    
    return container

# -------------------------
# Word Counter
# -------------------------

def create_word_counter():
    header = widgets.HTML(value="<h2>Word Counter</h2>")
    
    text_area = widgets.Textarea(
        description='Text:',
        placeholder='Enter text to count words',
        layout=widgets.Layout(width='100%', height='150px')
    )
    
    result_area = widgets.HTML(value="")
    
    count_button = widgets.Button(description="Count Words")
    
    def on_count(b):
        text = text_area.value
        
        if not text:
            result_area.value = "<p style='color:orange'>Please enter some text</p>"
            return
            
        words = text.split()
        word_count = len(words)
        char_count = len(text)
        char_no_spaces = len(text.replace(" ", ""))
        
        result_area.value = f"""
        <div style='padding:10px; background-color:#f0f8ff; border-radius:5px;'>
            <p><b>Word Count:</b> {word_count}</p>
            <p><b>Character Count (with spaces):</b> {char_count}</p>
            <p><b>Character Count (without spaces):</b> {char_no_spaces}</p>
        </div>
        """
    
    count_button.on_click(on_count)
    
    container = widgets.VBox([
        header,
        text_area,
        count_button,
        result_area
    ])
    
    return container

# -------------------------
# Wordle Game
# -------------------------

def create_wordle_game():
    header = widgets.HTML(value="<h2>Wordle Game</h2>")
    
    # Game data
    game_data = {
        'word': '',
        'attempts': 6,
        'guesses': [],
        'game_active': False,
        'word_length': 5,
        'custom_words': []
    }
    
    # Game controls
    word_length_selector = widgets.IntSlider(
        description='Word Length:',
        min=3, max=8, value=5,
        style={'description_width': 'initial'}
    )
    
    custom_word_input = widgets.Text(
        description='Custom Word:',
        placeholder='Add your own word',
        style={'description_width': 'initial'}
    )
    
    add_word_button = widgets.Button(description="Add Word")
    
    def on_add_word(b):
        word = custom_word_input.value.lower()
        
        if not word:
            game_status.value = "<p style='color:orange'>Please enter a word</p>"
            return
            
        if len(word) != game_data['word_length']:
            game_status.value = f"<p style='color:red'>Word must be {game_data['word_length']} letters long</p>"
            return
            
        if not word.isalpha():
            game_status.value = "<p style='color:red'>Word must contain only letters</p>"
            return
            
        game_data['custom_words'].append(word)
        custom_word_input.value = ""
        game_status.value = f"<p style='color:green'>Added word: {word}</p>"
    
    add_word_button.on_click(on_add_word)
    
    # Start game button
    start_button = widgets.Button(description="Start New Game")
    
    guess_input = widgets.Text(
        description='Guess:',
        placeholder='Enter your guess'
    )
    
    guess_button = widgets.Button(description="Submit Guess")
    
    game_status = widgets.HTML(value="<p>Set up the game and press Start</p>")
    
    guesses_display = widgets.HTML(value="")
    
    # On word length change
    def on_word_length_change(change):
        game_data['word_length'] = change['new']
        # Clear custom words if length changes
        game_data['custom_words'] = []
    
    word_length_selector.observe(on_word_length_change, names='value')
    
    # Random word generation (simulated)
    def get_random_word(length):
        # Default word lists by length
        word_lists = {
            3: ["cat", "dog", "hat", "run", "sun", "fun", "box", "fox", "pen"],
            4: ["home", "cats", "dogs", "jump", "love", "hate", "time", "game"],
            5: ["house", "chair", "table", "music", "piano", "world", "earth"],
            6: ["pandas", "python", "coding", "laptop", "screen", "flower"],
            7: ["program", "desktop", "monitor", "kitchen", "bedroom"],
            8: ["computer", "notebook", "textbook", "building", "baseball"]
        }
        
        # Use custom words if available
        if game_data['custom_words']:
            return random.choice(game_data['custom_words'])
            
        # Otherwise use default words
        if length in word_lists:
            return random.choice(word_lists[length])
        else:
            # Fallback
            return "python"[:length]
    
    # Start new game
    def on_start_game(b):
        game_data['word'] = get_random_word(game_data['word_length'])
        game_data['attempts'] = 6
        game_data['guesses'] = []
        game_data['game_active'] = True
        
        game_status.value = f"<p>I'm thinking of a {game_data['word_length']}-letter word. You have {game_data['attempts']} attempts.</p>"
        guesses_display.value = ""
        guess_input.value = ""
    
    start_button.on_click(on_start_game)
    
    # Submit guess
    def on_guess(b):
        if not game_data['game_active']:
            game_status.value = "<p style='color:orange'>Please start a new game first</p>"
            return
            
        guess = guess_input.value.lower()
        
        if len(guess) != len(game_data['word']):
            game_status.value = f"<p style='color:red'>Your guess must be {len(game_data['word'])} letters long</p>"
            return
            
        if not guess.isalpha():
            game_status.value = "<p style='color:red'>Your guess must contain only letters</p>"
            return
            
        game_data['guesses'].append(guess)
        game_data['attempts'] -= 1
        
        # Check if correct
        if guess == game_data['word']:
            game_status.value = f"<p style='color:green'>Congratulations! You guessed the word: {game_data['word']}</p>"
            game_data['game_active'] = False
        else:
            # Generate result string
            result = ""
            for i in range(len(game_data['word'])):
                if i < len(guess):
                    if guess[i] == game_data['word'][i]:
                        result += f"<span style='color:green; font-weight:bold;'>{guess[i].upper()}</span> "
                    elif guess[i] in game_data['word']:
                        result += f"<span style='color:orange; font-weight:bold;'>{guess[i].lower()}</span> "
                    else:
                        result += f"<span style='color:gray;'>_</span> "
                else:
                    result += "_ "
                    
            # Update guesses display
            current_display = guesses_display.value
            guesses_display.value = f"{current_display}<p>Guess {6 - game_data['attempts']}: {guess} → {result}</p>"
            
            # Check if out of attempts
            if game_data['attempts'] <= 0:
                game_status.value = f"<p style='color:red'>Game over! The word was: {game_data['word']}</p>"
                game_data['game_active'] = False
            else:
                game_status.value = f"<p>You have {game_data['attempts']} attempts remaining</p>"
                
        guess_input.value = ""
    
    guess_button.on_click(on_guess)
    
    container = widgets.VBox([
        header,
        widgets.HBox([word_length_selector]),
        widgets.HBox([custom_word_input, add_word_button]),
        start_button,
        game_status,
        widgets.HBox([guess_input, guess_button]),
        guesses_display
    ])
    
    return container

# -------------------------
# Main Application Setup
# -------------------------

def on_app_select(change):
    app_name = change['new']
    
    with content_area:
        content_area.clear_output()
        
        if app_name == 'ATM Machine':
            display(create_atm_app())
        elif app_name == 'Calculator':
            display(create_calculator_app())
        elif app_name == 'Currency Exchanger':
            display(create_currency_exchanger())
        elif app_name == 'Even/Odd Checker':
            display(create_even_odd_checker())
        elif app_name == 'Grade Tracker':
            display(create_grade_tracker())
        elif app_name == 'Multiplication Table':
            display(create_multiplication_table())
        elif app_name == 'Number Guessing Game':
            display(create_number_guessing_game())
        elif app_name == 'Palindrome Checker':
            display(create_palindrome_checker())
        elif app_name == 'Password Checker':
            display(create_password_checker())
        elif app_name == 'Rock Paper Scissors':
            display(create_rock_paper_scissors())
        elif app_name == 'To-Do List':
            display(create_todo_list())
        elif app_name == 'Weather App':
            display(create_weather_app())
        elif app_name == 'Word Counter':
            display(create_word_counter())
        elif app_name == 'Wordle Game':
            display(create_wordle_game())
        elif app_name == 'Select an Application':
            display(widgets.HTML(value="<h3>Welcome to Python Applications</h3><p>Please select an application from the dropdown menu to get started.</p>"))

# Connect app selector to handler
app_selector.observe(on_app_select, names='value')

# Set API Keys section
api_keys_header = widgets.HTML(value="<h3>Manage API Keys</h3>")

weather_api_key_input = widgets.Password(
    description='Weather API Key:',
    placeholder='Enter your API key from weatherapi.com',
    style={'description_width': 'initial'},
    value=app_data["api_keys"]["weather"]
)

currency_api_key_input = widgets.Password(
    description='Currency API Key:',
    placeholder='Enter your API key from currencyapi.com',
    style={'description_width': 'initial'},
    value=app_data["api_keys"]["currency"]
)

def on_api_key_save(b):
    app_data["api_keys"]["weather"] = weather_api_key_input.value
    app_data["api_keys"]["currency"] = currency_api_key_input.value
    api_key_status.value = "<p style='color:green'>API keys saved successfully!</p>"

api_key_save_button = widgets.Button(description="Save API Keys")
api_key_save_button.on_click(on_api_key_save)

api_key_status = widgets.HTML(value="")

api_keys_section = widgets.VBox([
    api_keys_header,
    widgets.HTML(value="<p>Set your API keys once and use them across all applications:</p>"),
    weather_api_key_input,
    currency_api_key_input,
    api_key_save_button,
    api_key_status
])

# Toggle API Keys section
show_api_keys = widgets.Button(
    description="Show/Hide API Keys",
    layout=widgets.Layout(width='150px')
)

api_keys_container = widgets.VBox([
    show_api_keys
])

def on_toggle_api_keys(b):
    if len(api_keys_container.children) == 1:
        # Show API keys section
        api_keys_container.children = [show_api_keys, api_keys_section]
    else:
        # Hide API keys section
        api_keys_container.children = [show_api_keys]

show_api_keys.on_click(on_toggle_api_keys)

# Main application container
app_container = widgets.VBox([
    app_header,
    api_keys_container,
    app_selector,
    content_area
])

# Display the application
display(app_container)